In [1]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Dense, Dropout, 
                                   BatchNormalization, Input, GlobalAveragePooling2D, 
                                   Concatenate, Multiply, Reshape)
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K
import numpy as np
import os
import cv2
import random
import albumentations as A
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from tensorflow.keras.optimizers import AdamW
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import seaborn as sns
import pickle

# %pip install optuna 
# %pip install optuna-integration[tfkeras]
import optuna
from optuna.integration import TFKerasPruningCallback


#*** Model Save is disanbled for testing purposes ***

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  

# === Configuration ===
config = {
    "epochs": 1,
    "is_config_batch_size_param": True,
    "batch_size": 200,
    "initial_lr": 0.001,
    "gpu_memory_limit": 45,
    "target_size": (480, 640),  # 2:3 ratio (width, height)
    "input_shape": (640, 480, 3), # (height, width, channels) for Keras
    "data_path": "Dataset/merged_SMOT_train",
    "csv_path": "processed_data/cleaned_metadata_short.csv",
    "train_set_csv": "Model/training8_customCNN_rgb_att_SMOT_aug_bay/training8_customCNN_rgb_att_SMOT_aug_bay_train_set.csv",
    "val_set_csv": "Model/training8_customCNN_rgb_att_SMOT_aug_bay/training8_customCNN_rgb_att_SMOT_aug_bay_validation_set.csv",
    "history_csv": "Model/training8_customCNN_rgb_att_SMOT_aug_bay/training8_customCNN_rgb_att_SMOT_aug_bay_history.csv",
    "best_model": "Model/training8_customCNN_rgb_att_SMOT_aug_bay/training8_customCNN_rgb_att_SMOT_aug_bay_best_model.keras",
    "label_encoder_path": "Model/training8_customCNN_rgb_att_SMOT_aug_bay/training8_customCNN_rgb_att_SMOT_aug_bay_label_encoder.npy",
    "color_channel": "",
    "save_dir": "Model/training8_customCNN_rgb_att_SMOT_aug_bay",
}

2025-05-15 02:51:19.849905: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-15 02:51:19.853551: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-15 02:51:19.943399: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-15 02:51:19.976990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747252280.039372    6538 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747252280.05

In [2]:
# === GPU Setup ===
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        policy = mixed_precision.Policy('float32')
        mixed_precision.set_global_policy(policy)
        
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.optimizer.set_jit(True)
        tf.config.threading.set_intra_op_parallelism_threads(8)
        tf.config.threading.set_inter_op_parallelism_threads(4)
    except RuntimeError as e:
        print(e)

# === Memory Management ===
def calculate_max_batch_size(model, input_shape, gpu_mem=24, default_batch=32, is_use_config_batch_size=config["is_config_batch_size_param"]):
    """Improved batch size calculator with error handling"""
    if is_use_config_batch_size:
        return default_batch
    try:
        params = model.count_params()
        
        last_dense = None
        for layer in reversed(model.layers):
            if isinstance(layer, tf.keras.layers.Dense):
                last_dense = layer
                if layer.name == 'features':  
                    break
        
        if last_dense is None:
            raise ValueError("No Dense layer found in model!")
        
        # Memory per sample 
        per_sample = (
            (params * 4) +                 
            (np.prod(input_shape) * last_dense.units * 4)  
        ) / (1024 ** 3)
        
        max_batch = int((gpu_mem - 3) / per_sample)
        return min(256, max_batch)  
    
    except Exception as e:
        print(f"Warning: Batch size estimation failed, using default={default_batch}. Error: {e}")
        return default_batch

def cleanup_gpu_memory():
    """Force clear GPU memory"""
    K.clear_session()
    tf.compat.v1.reset_default_graph()
    if tf.config.list_physical_devices('GPU'):
        try:
            for gpu in tf.config.list_physical_devices('GPU'):
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError:
            pass

# === Data Pipeline ===
def load_and_preprocess_data(random_state=42, save_splits=True):
    """Load and split data with fixed random state for reproducibility"""
    df = pd.read_csv(config["csv_path"])
    
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    print(f"Label classes: {le.classes_}")
    
    with open(config['label_encoder_path'], 'wb') as f:
        np.save(f, le.classes_)
    
    train_df, val_df = train_test_split(
        df, 
        test_size=0.2, 
        stratify=df['label'],
        random_state=random_state,
    )
    
    if save_splits:
        train_df.to_csv(config['train_set_csv'], index=False)
        val_df.to_csv(config['val_set_csv'], index=False)
    
    return train_df, val_df, le


2025-05-15 02:51:23.470884: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
class RiceDataGenerator(Sequence):
    def __init__(self, df, base_path, batch_size=32, target_size=(480, 640), shuffle=False, debug=False, config=None, **kwargs):
        super().__init__(**kwargs)
        self.df = df.reset_index(drop=True)
        self.base_path = base_path
        self.batch_size = batch_size
        self.target_size = target_size  
        self.shuffle = shuffle
        self.debug = debug
        self.indices = np.arange(len(df))
        self.config = config if config else {}
        
        self.aug = A.Compose(config["augmentation"])
        
        if shuffle:
            np.random.shuffle(self.indices)
            
        if self.debug:
            self.visualize_samples()    
            

    def visualize_samples(self):        
        try:
            row = self.df.iloc[0]
            img = self._load_image(row['image_id'], row['label'])
            augmented = self.aug(image=img)
            
            plt.figure(figsize=(12, 6))
            
            # original
            plt.subplot(1, 2, 1)
            plt.imshow(img)
            plt.title(f"Original\nShape: {img.shape}")
            
            # augmented
            plt.subplot(1, 2, 2)
            plt.imshow(augmented['image'])
            plt.title(f"Augmented\nShape: {augmented['image'].shape}")
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"Visualization failed for {row['image_id']}: {str(e)}")
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def _load_image(self, image_id, label):
        img_path = os.path.join(
            self.base_path,
            label,
            f"{os.path.splitext(image_id)[0]}.jpg"
        )
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Image not found at {img_path}")
        return img
    
    def __getitem__(self, idx):
        batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_df = self.df.iloc[batch_indices]
        
        X = np.zeros((len(batch_df), self.target_size[1], self.target_size[0], 3), dtype=np.float32) #(batch, height, width, channels)
        y = np.zeros((len(batch_df),), dtype=np.int32)
        
        for i, (_, row) in enumerate(batch_df.iterrows()):
            try:
                img = self._load_image(row['image_id'], row['label'])
                augmented = self.aug(image=img)
                X[i] = augmented['image'] / 255.0
                y[i] = row['label_encoded']
            except Exception as e:
                print(f"Error loading {row['image_id']}: {str(e)}")
                X[i] = np.zeros((self.target_size[1], self.target_size[0], 3), dtype=np.float32) #(batch, height, width, channels)
                y[i] = -1
                
        valid = y != -1
        return X[valid], y[valid]


In [4]:
# === Model Architecture ===
def se_block(input_tensor, ratio=16):
    channels = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(channels // ratio, activation="relu")(se)
    se = Dense(channels, activation="sigmoid")(se)
    return Multiply()([input_tensor, se])

def create_customCNN(input_shape, num_classes):    
    inputs = Input(shape=input_shape, dtype=tf.float32) 
     
    # Initial feature extraction
    x = Conv2D(96, (7,7), strides=2, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3), strides=2)(x)
    
    x = se_block(x)

    # Intermediate layers
    x = Conv2D(256, (5,5), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3), strides=2)(x)
    
    # Parallel paths
    branch1 = Conv2D(384, (3,3), activation='relu', padding='same')(x)
    branch2 = Conv2D(384, (3,3), dilation_rate=2, activation='relu', padding='same')(x)
    x = Concatenate()([branch1, branch2])
        
    # Final classification head
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu', name='features')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax', dtype=tf.float32)(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# === Training ===
def train(config=None):
    cleanup_gpu_memory()
    
    try:
        train_df, val_df, le = load_and_preprocess_data(random_state=42)
        num_classes = len(le.classes_)
        print("Classes: ", num_classes)
        
        # Create the mopdel
        input_shape = config["input_shape"] 
        model = create_customCNN(input_shape, num_classes)
        
        # Find optimal batch size
        cleanup_gpu_memory()
        optimized_batch_size = calculate_max_batch_size(
                                    model, 
                                    input_shape=config["input_shape"],
                                    gpu_mem=config["gpu_memory_limit"],
                                    default_batch=config["batch_size"],
                                )
        
        print(f"\n=== Training Configuration ===")
        print(f"Batch size: {optimized_batch_size}")
        print(f"Input size: {config['target_size']}")
        print(f"Classes: {num_classes}")
        print(f"GPU Memory: {config['gpu_memory_limit']}GB\n")
        print(f"Model input shape: {model.input_shape}")
        
        # Create generators for training and validation
        train_gen = RiceDataGenerator(
            df=train_df,
            base_path=config["data_path"],
            batch_size=optimized_batch_size,
            target_size=config["target_size"],
            shuffle=False,
            debug=True,
            config=config
        )
        
        val_gen = RiceDataGenerator(
            df=val_df,
            base_path=config["data_path"],
            batch_size=optimized_batch_size,
            target_size=config["target_size"],
            shuffle=False,
            debug=False,
            config=config
        )
        
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        sample_batch = train_gen[0]
        print(f"Generator output shape: {sample_batch[0].shape}")
        print(f"Model input shape: {model.input_shape}")
        
        # Train
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=config["epochs"],
            callbacks=[
                    tf.keras.callbacks.EarlyStopping(
                    monitor='val_accuracy',
                    patience=5, 
                    mode='max',
                    restore_best_weights=True  
                ),
                tf.keras.callbacks.ModelCheckpoint(
                    config["best_model"],  
                    save_weights_only=False,
                    monitor='val_accuracy',
                    save_best_only=True 
                ),
                    tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_accuracy',
                    factor=0.5,  # Halve Learn Rate
                    patience=3,
                    mode='max'
                )
            ]
        )
        
        plt.figure(figsize=(12, 6))
        
        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Val Accuracy')
        plt.axhline(y=max(history.history['val_accuracy']), color='r', linestyle='--', label='Best Val Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        
        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.axhline(y=min(history.history['val_loss']), color='r', linestyle='--', label='Best Val Loss')
        plt.title('Model Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
        
        return model, history
        
    except Exception as e:
        print(f"Training failed: {e}")
        cleanup_gpu_memory()
        raise

In [6]:

def objective(trial):
    # Hyperparameters to optimize
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.7)
    
    # Update config with suggested values
    config.update({
        "initial_lr": lr,
        "batch_size": batch_size,
        "dropout_rate": dropout_rate,
    })
    
    # Load data and train (use your existing `train()` function)
    cleanup_gpu_memory()
    model, history = train(config)
    
    # Return validation accuracy (what we want to maximize)
    return max(history.history["val_accuracy"])

In [7]:
def create_objective(config, train_df, val_df, num_classes):
    def objective(trial):
        # Hyperparameters to optimize
        lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
        batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
        dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.7)
        conv_filters = trial.suggest_categorical("conv_filters", [64, 96, 128])
        
        # Update config with suggested values
        current_config = config.copy()
        current_config.update({
            "initial_lr": lr,
            "batch_size": batch_size,
            "dropout_rate": dropout_rate,
        })
        
        # Create model with dynamic architecture
        def create_model():
            inputs = Input(shape=current_config["input_shape"], dtype=tf.float32)
            x = Conv2D(conv_filters, (7,7), strides=2, activation='relu', padding='same')(inputs)
            x = BatchNormalization()(x)
            x = MaxPooling2D((3,3), strides=2)(x)
            x = se_block(x)

            # Intermediate layers
            x = Conv2D(256, (5,5), activation='relu', padding='same')(x)
            x = BatchNormalization()(x)
            x = MaxPooling2D((3,3), strides=2)(x)
            
            # Parallel paths
            branch1 = Conv2D(384, (3,3), activation='relu', padding='same')(x)
            branch2 = Conv2D(384, (3,3), dilation_rate=2, activation='relu', padding='same')(x)
            x = Concatenate()([branch1, branch2])
                
            # Final classification head
            x = GlobalAveragePooling2D()(x)
            x = Dense(512, activation='relu', name='features')(x)
            x = Dropout(0.5)(x)
            outputs = Dense(num_classes, activation='softmax', dtype=tf.float32)(x)
            
            return tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Create data generators
        train_gen = RiceDataGenerator(
            df=train_df,
            base_path=current_config["data_path"],
            batch_size=batch_size,
            target_size=current_config["target_size"],
            config=current_config
        )
        
        val_gen = RiceDataGenerator(
            df=val_df,
            base_path=current_config["data_path"],
            batch_size=batch_size,
            target_size=current_config["target_size"],
            config=current_config
        )
        
        # Create and compile model
        model = create_model()
        model.compile(
            optimizer=AdamW(learning_rate=lr),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Train with pruning
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=current_config["epochs"],
            callbacks=[
                TFKerasPruningCallback(trial, "val_accuracy"),
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_accuracy',
                    patience=5,
                    restore_best_weights=True
                )
            ],
            verbose=0
        )
        
        return max(history.history['val_accuracy'])
    
    return objective

In [8]:
def optimize_hyperparameters(config, n_trials=50):
    cleanup_gpu_memory()
    
    # Load data
    train_df, val_df, le = load_and_preprocess_data()
    num_classes = len(le.classes_)
    
    # Create objective
    objective = create_objective(config, train_df, val_df, num_classes)
    
    # Run optimization
    study = optuna.create_study(
        direction="maximize",
        pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5)
    )
    study.optimize(objective, n_trials=n_trials)
    
    # Save best parameters
    best_params = study.best_params
    print("Best trial:")
    print(f"  Value (val_accuracy): {study.best_value}")
    print("  Params: ")
    for key, value in best_params.items():
        print(f"    {key}: {value}")
    
    return best_params

In [ ]:
config_1 = {
    **config, 
    "target_size": (255, 255),
    "input_shape": (255, 255, 3),
    "augmentation": [
        A.Resize(width=255, height=255),
        A.HueSaturationValue(p=0.5),
        A.CoarseDropout(num_holes_range=[5, 10], hole_height_range=[0.01, 0.02], hole_width_range=[0.01, 0.02], max_holes=3, max_height=1, max_width=1),

    ]
}

# Run optimization
best_params_1 = optimize_hyperparameters(config_1, n_trials=10)

# Update config with best parameters
optimized_config_1 = config_1.copy()
optimized_config_1.update(best_params_1)

# Train final model with optimized parameters
final_model_1, final_history = train(optimized_config_1)

# Save optimized model
final_model_1.save(os.path.join(config["save_dir"], 'optimized_model_1.keras'))

/tmp/ipykernel_6538/574296162.py:8: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(num_holes_range=[5, 10], hole_height_range=[0.01, 0.02], hole_width_range=[0.01, 0.02], max_holes=3, max_height=1, max_width=1),
[I 2025-05-15 02:51:23,631] A new study created in memory with name: no-name-23d0ee51-8fa4-428f-bcb0-7059b8682dc1


Label classes: ['bacterial_leaf_blight' 'bacterial_panicle_blight' 'blast' 'brown_spot'
 'dead_heart' 'downy_mildew']


[I 2025-05-15 02:51:26,889] Trial 0 finished with value: 0.125 and parameters: {'lr': 0.00010446064000635346, 'batch_size': 128, 'dropout_rate': 0.597055350730155, 'conv_filters': 64}. Best is trial 0 with value: 0.125.
[I 2025-05-15 02:51:30,743] Trial 1 finished with value: 0.125 and parameters: {'lr': 0.0003256527343062363, 'batch_size': 64, 'dropout_rate': 0.3249569112855539, 'conv_filters': 128}. Best is trial 0 with value: 0.125.


[I 2025-05-15 02:51:34,564] Trial 2 finished with value: 0.25 and parameters: {'lr': 1.5182200682270374e-05, 'batch_size': 128, 'dropout_rate': 0.6378327526015831, 'conv_filters': 128}. Best is trial 2 with value: 0.25.
[I 2025-05-15 02:51:37,912] Trial 3 finished with value: 0.125 and parameters: {'lr': 0.00011572009729644789, 'batch_size': 64, 'dropout_rate': 0.5182607165409326, 'conv_filters': 96}. Best is trial 2 with value: 0.25.
[I 2025-05-15 02:51:40,926] Trial 4 finished with value: 0.125 and parameters: {'lr': 0.00013632687139692799, 'batch_size': 128, 'dropout_rate': 0.5470709174837067, 'conv_filters': 64}. Best is trial 2 with value: 0.25.
[I 2025-05-15 02:51:44,267] Trial 5 finished with value: 0.25 and parameters: {'lr': 0.0006171279950365855, 'batch_size': 64, 'dropout_rate': 0.3589953608110795, 'conv_filters': 64}. Best is trial 2 with value: 0.25.
[I 2025-05-15 02:51:48,077] Trial 6 finished with value: 0.125 and parameters: {'lr': 3.243206319839892e-05, 'batch_size': 6